In [ ]:
#!pip install gurobipy pandas openpyxl
#!pip install gdown


   ----------------------------------------  0/12 [urllib3]
   --- ------------------------------------  1/12 [typing-extensions]
   ------ ---------------------------------  2/12 [tqdm]
   ------ ---------------------------------  2/12 [tqdm]
   ---------- -----------------------------  3/12 [soupsieve]
   ---------------- -----------------------  5/12 [idna]
   ----------------------- ----------------  7/12 [charset_normalizer]
   ------------------------------ ---------  9/12 [requests]
   --------------------------------- ------ 10/12 [beautifulsoup4]
   --------------------------------- ------ 10/12 [beautifulsoup4]
   --------------------------------- ------ 10/12 [beautifulsoup4]
   --------------------------------- ------ 10/12 [beautifulsoup4]
   ---------------------------------------- 12/12 [gdown]



In [1]:
import pandas as pd
import gdown
from gurobipy import Model, GRB, quicksum

In [2]:
#Descargamos el archivo desde Drive
file_id = '1BNagy0v4Nwg8dCyvZdS9rZOaR2-L9ROL'
output = 'INPUT MODELO OPTIMIZACIÓN.xlsx'
gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1BNagy0v4Nwg8dCyvZdS9rZOaR2-L9ROL
To: e:\proyectos\optimizacion_TDG\tdg2_mcd\INPUT MODELO OPTIMIZACIÓN.xlsx
100%|██████████| 277k/277k [00:00<00:00, 1.25MB/s]


'INPUT MODELO OPTIMIZACIÓN.xlsx'

In [ ]:
# ==== PREPARACIÓN DE DICCIONARIOS PARA CONSTRUIR CONJUNTOS Y PARÁMETROS ===
dias = 6 #Días de corrida del modelo
# === 1. LEER DATOS DE ENTRADA ===
archivo = "INPUT MODELO OPTIMIZACIÓN.xlsx"
df_granjas = pd.read_excel(archivo, sheet_name="Input_Granjas (2)")
df_demanda = pd.read_excel(archivo, sheet_name="Input_Demanda (2)", header=4)
df_raw = pd.read_excel(archivo, sheet_name="Input_Demanda (2)", header=None)

# === PARÁMETROS ===
W, ADG, CAT, N, TIPO = {}, {}, {}, {}, {}

for _, row in df_granjas.iterrows():
    g, p, s = row["GRANJA"], row["GALPON"], row["SEXO"]
    N[(g, p, s)] = row["SALDO"]
    TIPO[(g, p, s)] = row["TIPO"]

    for d in range(1, 1 + dias):
        col_w = 'WEIGTH' if d == 1 else f'WEIGTH{d}'
        col_adg = 'ADG' if d == 1 else f'ADG{d}'
        col_cat = 'CAT PESO' if d == 1 else f'CAT PESO{d}'

        if pd.notnull(row[col_w]): W[(g, p, s, d)] = float(row[col_w])
        if pd.notnull(row[col_adg]): ADG[(g, p, s, d)] = float(row[col_adg])
        if pd.notnull(row[col_cat]): CAT[(g, p, s, d)] = row[col_cat].strip().upper()

# === DEMANDA Q[tipo, cat, d] ===
Q = {}
tipos_validos = ["CAMPESINO", "BLANCO"]
cats_validas = ["PEQUEÑO", "MEDIANO", "GRANDE"]

for _, row in df_demanda.iterrows():
    t, c = row["Tipo"], row["Categoria Peso"]
    if t in tipos_validos and c in cats_validas:
        for d in range(1, 1 + dias):
            val = row.get(f"Day{d}")
            if pd.notnull(val):
                Q[(t, c, d)] = val

# === MAPD y TOT ===
MAPD = {}

for i in range(13, 13 + dias):
    semana = df_raw.iloc[i, 1]
    dia = df_raw.iloc[i, 2]
    if pd.notnull(semana) and pd.notnull(dia):
        MAPD[int(dia)] = int(semana)

TOT = {}
val_tot = df_raw.iloc[15, 5]
if pd.notnull(val_tot): TOT[1] = float(val_tot)

tol = float(df_raw.iloc[1, 2])  # Tolerancia


In [46]:
# === 2. DEFINICIÓN DE CONJUNTOS ===


# Conjunto de grupos: (granja, galpón, sexo)
GP = list(N.keys())

# Conjunto de días en el horizonte (por ejemplo 1 a 6)
D = sorted(set(d for (_, _, _, d) in W.keys()))

# Conjunto de tipos de pollo
T = sorted(set(TIPO.values()))

# Conjunto de categorías de peso
C = sorted(set(c for (_, _, _, _), c in CAT.items()))

# Conjunto de semanas (extraído del mapeo día → semana)
WEEK = sorted(set(MAPD[d] for d in MAPD))


In [47]:
# === 3. CREACION DEL MODELO ===

# === MODELO VACÍO ===
model = Model("modelo_cosecha_pollos")

# === VARIABLES ===

# x[g, p, s, d] = 1 si el grupo se cosecha el día d
x = model.addVars(GP, D, vtype=GRB.BINARY, name="x")

# dev[t, c, d] = desviación absoluta entre cosecha y demanda para ese tipo, categoría y día
dev = model.addVars(T, C, D, vtype=GRB.CONTINUOUS, lb=0, name="dev")

In [48]:
# === 4. FUNCIÓN OBJETIVO ===

model.setObjective(
    quicksum(dev[t, c, d] for t in T for c in C for d in D),
    GRB.MINIMIZE
)


In [ ]:
# === 5. PLANTEAMIENTO DE LAS RESTRICCIONES ===

# === R1: Cada grupo se cosecha 1 sola vez

for g, p, s in GP:
    model.addConstr(
        quicksum(x[g, p, s, d] for d in D) <= 1,
        name=f"R1_una_vez_{g}_{p}_{s}"
    )


In [ ]:

# === R2: la diferencia absoluta entre lo cosechado y la demanda debe ser menor a la desviacion ===

#for t in T:
    #for c in C:
        #for d in D:
            #q_val = Q.get((t, c, d), 0)
            #if q_val > 0:
                #model.addConstr(
                    #dev[t, c, d] <= 0.15 * q_val,
                    #name=f"R2_limite_10pct_{t}_{c}_{d}"
                #)


for t in T:
    for c in C:
        for d in D:
            cantidad_real = quicksum(
                N[gps] * x[gps + (d,)] # Asegura que el grupo haya sido cosechado, y agarra la cantidad del grupo (N)
                for gps in GP
                if TIPO[gps] == t and CAT.get(gps + (d,), None) == c #Asegura que las categorias y tipos coincidan
            )
            if (t, c, d) in Q:
                model.addConstr(cantidad_real - Q[t, c, d] <= dev[t, c, d], # Modelado de la restricción en absoluto
                                name=f"R2_sup_{t}_{c}_{d}")
                model.addConstr(Q[t, c, d] - cantidad_real <= dev[t, c, d], # Modelado de la restricción en absoluto
                                name=f"R2_inf_{t}_{c}_{d}")

In [ ]:
# === R3: Para los grupos mixtos, se prioriza cosechar las hembras sobre los machos ===
galpones = set((g, p) for g, p, s in GP)

for g, p in galpones:
    if ((g, p, 'HEMBRA') in GP) and ((g, p, 'MACHO') in GP):
        for d in D:
            if d >= 5: #Solo tiene sentido si han transcurrido más de 4 días 
                suma_hembras_previos = quicksum(
                    x[g, p, 'HEMBRA', dp]
                    for dp in range(d - 4, d) if dp in D #Valida si las hembras de un galpon mixto se cosechan durante 4 días anteriores
                )
                model.addConstr(
                    x[g, p, 'MACHO', d] <= 1 - suma_hembras_previos,
                    name=f"R3_prioridad_{g}_{p}_{d}" #Evita que se cosechen los machos si no han pasado 4 días o más desde la cosecha de hembras
                )

# === R4: Asegura que la cosecha de la semana esté dentro del rango de tolerancia de la demanda total para esa semana ===

for w in WEEK:
    dias_semana = [d for d in D if MAPD.get(d) == w]
    
    suma_cosechada = quicksum(
        N[gps] * x[gps + (d,)] #Computa cuantos pollos sales por grupo cosechado
        for gps in GP
        for d in dias_semana
    )
    
    if w in TOT:
        model.addConstr(
            suma_cosechada >= (1 - tol) * TOT[w], #Límite inferior de la demanda total semanal
            name=f"R5_inferior_{w}"
        )
        model.addConstr(
            suma_cosechada <= (1 + tol) * TOT[w], #Límite superior de la demanda total semanal
            name=f"R5_superior_{w}"
        )


# === R5: No cosechar pesos por encima de 3.5 Kg ===
for (g, p, s, d), peso in W.items(): #itera sobre cada arreglo de granja, galpon, sexo y día para asegurar que el peso no se pase de 3.5 Kg
    if peso > 3.5:
        model.addConstr(
            x[g, p, s, d] == 0,
            name=f"R_peso_maximo_{g}_{p}_{s}_{d}"
        )


# === R6: El mínimo peso del pollo campesino debe ser 3 Kg ===

for (g, p, s, d), peso in W.items():
    if TIPO.get((g, p, s)) == "CAMPESINO" and peso < 3.0:
        model.addConstr(
            x[g, p, s, d] == 0,
            name=f"R_peso_min_campesino_{g}_{p}_{s}_{d}"
        )

# === R7: Solo cosechar los grupos con demanda ===

for (g, p, s, d), cat in CAT.items():
    tipo = TIPO.get((g, p, s))
    if (tipo, cat, d) not in Q:
        model.addConstr(
            x[g, p, s, d] == 0,
            name=f"R_sin_demanda_{g}_{p}_{s}_{d}"
        )


In [52]:
# Resolver el modelo
model.optimize()


Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2680166 - for non-commercial use only - registered to 10___@u.icesi.edu.co
Optimize a model with 3306 rows, 1758 columns and 17288 nonzeros
Model fingerprint: 0xaa39c84a
Variable types: 48 continuous, 1710 integer (1710 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 24 rows and 12 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [40]:
#Revisar el estado del modelo de optimizacion

if model.status == GRB.OPTIMAL:
    print("\n✅ Solución óptima encontrada.")
elif model.status == GRB.INFEASIBLE:
    print("\n❌ El modelo es infactible.")
    model.computeIIS()
    model.write("modelo_infactible.ilp")
elif model.status == GRB.UNBOUNDED:
    print("\n❌ El modelo no está acotado.")
else:
    print(f"\n⚠️ El modelo terminó con estado: {model.status}")



✅ Solución óptima encontrada.


In [41]:
#Resultados de la cosecha

print("\n📦 Grupos cosechados:")
for g, p, s in GP:
    for d in D:
        var = x[g, p, s, d]
        if var.X > 0.5:
            print(f"Grupo ({g}, {p}, {s}) cosechado el día {d}")



📦 Grupos cosechados:
Grupo (MAO, 5, M) cosechado el día 5
Grupo (MAO, 6, M) cosechado el día 5
Grupo (LA CHAMBA, 1, M) cosechado el día 1
Grupo (LA CHAMBA, 2, M) cosechado el día 3
Grupo (LA CHAMBA, 3, M) cosechado el día 2
Grupo (LA CHAMBA, 4, M) cosechado el día 2
Grupo (CHICO, 3, M) cosechado el día 2
Grupo (RANCHO ALEGRE, 1, M) cosechado el día 1
Grupo (RANCHO ALEGRE, 2, M) cosechado el día 4
Grupo (RANCHO ALEGRE, 3, M) cosechado el día 3
Grupo (RANCHO ALEGRE, 4, M) cosechado el día 2
Grupo (POTOCO, 1, M) cosechado el día 4
Grupo (POTOCO, 2, M) cosechado el día 6
Grupo (POTOCO, 3, M) cosechado el día 6
Grupo (POTOCO, 4, M) cosechado el día 6
Grupo (POTOCO, 5, M) cosechado el día 6
Grupo (POTOCO, 6, M) cosechado el día 4
Grupo (POTOCO, 7, M) cosechado el día 6
Grupo (POTOCO, 8, M) cosechado el día 5
Grupo (POTOCO, 9, M) cosechado el día 3
Grupo (ARANJUEZ, 1, M) cosechado el día 6
Grupo (ARANJUEZ, 4, M) cosechado el día 4
Grupo (ARANJUEZ, 5, M) cosechado el día 4
Grupo (ARANJUEZ, 6,

In [42]:
#Desviaciones presentadas

print("\n📉 Desviaciones en el cumplimiento del mix diario:")
for t in T:
    for c in C:
        for d in D:
            val = dev[t, c, d].X
            if val > 0.01:
                print(f"Tipo {t}, Categoría {c}, Día {d}: desviación = {val:.0f} aves")



📉 Desviaciones en el cumplimiento del mix diario:
Tipo BLANCO, Categoría GRANDE, Día 1: desviación = 3514 aves
Tipo BLANCO, Categoría GRANDE, Día 2: desviación = 1936 aves
Tipo BLANCO, Categoría GRANDE, Día 3: desviación = 6514 aves
Tipo BLANCO, Categoría GRANDE, Día 4: desviación = 26 aves
Tipo BLANCO, Categoría GRANDE, Día 5: desviación = 756 aves
Tipo BLANCO, Categoría GRANDE, Día 6: desviación = 1037 aves
Tipo BLANCO, Categoría GRANDE, Día 7: desviación = 11536 aves
Tipo BLANCO, Categoría GRANDE, Día 8: desviación = 11536 aves
Tipo BLANCO, Categoría GRANDE, Día 9: desviación = 11536 aves
Tipo BLANCO, Categoría GRANDE, Día 10: desviación = 11536 aves
Tipo BLANCO, Categoría GRANDE, Día 11: desviación = 11536 aves
Tipo BLANCO, Categoría GRANDE, Día 12: desviación = 11536 aves
Tipo BLANCO, Categoría MEDIANO, Día 1: desviación = 72266 aves
Tipo BLANCO, Categoría MEDIANO, Día 2: desviación = 72266 aves
Tipo BLANCO, Categoría MEDIANO, Día 3: desviación = 66626 aves
Tipo BLANCO, Categoría

In [43]:
#Guardar resultados en un CSV
import csv

with open("resultados_cosecha.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Granja", "Galpón", "Sexo", "Día", "Cosechado (1/0)"])
    for g, p, s in GP:
        for d in D:
            writer.writerow([g, p, s, d, int(x[g, p, s, d].X > 0.5)])


In [44]:
import csv

with open("resultados_cosecha2.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Granja", "Galpón", "Sexo", "Día", "Tipo", "Categoría", "Cosechado (1/0)", "Cantidad cosechada"])

    for g, p, s in GP:
        tipo = TIPO[(g, p, s)]
        for d in D:
            cat = CAT.get((g, p, s, d), "NA")
            cosechado = int(x[g, p, s, d].X > 0.5)
            cantidad = int(N[(g, p, s)] * x[g, p, s, d].X + 0.5) if cosechado else 0

            writer.writerow([g, p, s, d, tipo, cat, cosechado, cantidad])
